### Import all the important libraries that are to be used.
**Note**: One of the images is a bank check of mine (Shakhaf). Kindly only use it only for evaluation! Thanks

In [1]:
# Author: Shakhaf
# Date: 28/01/2021
import numpy as np
import re
import cv2
import pytesseract
import os 

import requests
from difflib import SequenceMatcher
from PIL import Image
pytesseract.pytesseract.tesseract_cmd = 'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'
# image = cv2.imread("checkscan_250x250.jpg")

### This converts the original image into image of 540 height and 1200 width.

In [2]:
filename = "example1.jpg"
size = 1200, 540
file_parts = os.path.splitext(filename)

outfile = file_parts[0] + '_250x250' + file_parts[1]
try:
    img = Image.open(filename)
    img = img.resize(size, Image.ANTIALIAS)
    img.save(outfile, 'PNG')
except IOError as e:
    print("An exception occured '%s'" %e)
image = cv2.imread(outfile)

In [3]:
# Method Name: crop_images
# Description: This function crops the images depinding on the coordinates given.
def crop_images(x,h,y,w):
    try:
        image = cv2.imread(outfile)
        crop_image = image[x:h, y:w]
        cv2.imshow('Image', crop_image)
        cv2.waitKey(0) 
        cv2.destroyAllWindows()
        text = pytesseract.image_to_string(crop_image)
        print(text)
    except (Exception) as error:
        text = "-"
        print("error in crop_images", error)
    return text

In [4]:
# Method Name: account_number
# Description: This function gets thethe ifsc code as per given on the Kotak Mahindra check. It cleans the unecessary text and get
#              only the required details.
def ifsc_code(x,h,y,w):
    try:
        text = crop_images(x, h, y, w)
        text = text.replace("\n", "")
        text = text.replace(" ", "") 
        substring1 = "IFSC"
        substring2 = "IFSCode"
        substring3 = "IFSCode"
        if substring2 in text :      
            text = text.split("IFSCode")
        if substring1 in text :
            text = text.split("IFSC")
        if substring3 in text :
            text = text.split("IFScode")
        print(text)
        if type(text) == list:
            for i in range(len(text)):
                text[i] = text[i].upper()
                text[i]= text[i].replace("\n", " ")
                text[i]= text[i].replace("\x0c", "")
                text[i] = text[i].replace(" ", "") 
            text = text[1][1:12]
    except (Exception) as error:
        text = "-"
        print("error in ifsc_code", error)
    return text
text = ifsc_code(0,200,0,900)
text = str(text)
print(text)

    
  
     

 

Plot No-4, Jawahar Nagar,
Mumbal- 400062 :
Maharashtra India =<Date
IFSC : KKBK0000643 Say

KotaksMatindra Bok

 

['PlotNo-4,JawaharNagar,Mumbal-400062:MaharashtraIndia=<Date', ':KKBK0000643SayKotaksMatindraBok\x0c']
KKBK0000643


### Here, we are using Razorpay IFSC Toolkit to fetch IFSC code. Razorpay IFSC Toolkit processes IFSC codes via their toolkit and return data set. 
- We could have extracted the the MICR number just as we extracted the bank name, address, ifsc code and account number, but there is a simplier way by calling the Razorpay API.
- There is one more reason why we did not extract it from the image and that is because the it consists of a different font that  cannot be read by a python library. The only way that I could think of it was by using opencv(countors) library. It was tried using open cv library but it was only able to extract under very precise optimal conditions.
- Hence we have used the Razorpay IFSC Toolkit  

**Note** : The below API will only run for a real IFSC code. Else it will throw an error

In [5]:
def get_bank_details(text):
    try:
        URL = "https://ifsc.razorpay.com/"
        result = {}
        result = requests.get(URL + text).json() 
        print(result)
    except (Exception) as error:
        text = "-"
        print("error in get_bank_details", error)
    return result

In [6]:
get_bank_details(text)

{'UPI': True, 'SWIFT': '', 'CENTRE': 'MUMBAI', 'CITY': 'MUMBAI', 'DISTRICT': 'GREATER BOMBAY', 'BRANCH': 'GOREGAON', 'STATE': 'MAHARASHTRA', 'IMPS': True, 'CONTACT': '67587970', 'ADDRESS': 'KOTAK MAHINDRA BANK LTD. EVERSHINE ANGAN, GR FLOOR, PLOT NO 4, JAWAHAR NAGAR, GOREGAON WEST, MUMBAI 400062', 'RTGS': True, 'NEFT': True, 'MICR': '400485045', 'BANK': 'Kotak Mahindra Bank', 'BANKCODE': 'KKBK', 'IFSC': 'KKBK0000643'}


{'UPI': True,
 'SWIFT': '',
 'CENTRE': 'MUMBAI',
 'CITY': 'MUMBAI',
 'DISTRICT': 'GREATER BOMBAY',
 'BRANCH': 'GOREGAON',
 'STATE': 'MAHARASHTRA',
 'IMPS': True,
 'CONTACT': '67587970',
 'ADDRESS': 'KOTAK MAHINDRA BANK LTD. EVERSHINE ANGAN, GR FLOOR, PLOT NO 4, JAWAHAR NAGAR, GOREGAON WEST, MUMBAI 400062',
 'RTGS': True,
 'NEFT': True,
 'MICR': '400485045',
 'BANK': 'Kotak Mahindra Bank',
 'BANKCODE': 'KKBK',
 'IFSC': 'KKBK0000643'}

If the ifsc code was not found in teh above crop above it can be found in the below cropped image

In [7]:
text = ifsc_code(200,400,0,1000)

8
g
&

ces

2 SESS SSS EERO a ee

 

8g&ces2SESSSSSEEROaee


In [8]:
import cv2
import numpy as np
import pytesseract

pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
def micr_number(outfile):
    # Load image, convert to HSV format, define lower/upper ranges, and perform
    # color segmentation to create a binary mask
    image = cv2.imread(outfile)
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower = np.array([0, 0, 0])
    upper = np.array([160, 160, 160])
    mask = cv2.inRange(hsv, lower, upper)

    # Create horizontal kernel and dilate to connect text characters
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
    dilate = cv2.dilate(mask, kernel, iterations=7)

    # Find contours and filter using aspect ratio
    # Remove non-text contours by filling in the contour
    cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    for c in cnts:
        x,y,w,h = cv2.boundingRect(c)
        ar = w / float(h)
        if ar < 5:
            cv2.drawContours(dilate, [c], -1, (0,0,0), -1)

    # Bitwise dilated image with mask, invert, then OCR
    result = 255 - cv2.bitwise_and(dilate, mask)
    data = pytesseract.image_to_string(result, lang='mcr',config='--psm 6')
    print(data)
    cv2.imshow('mask', mask)
    cv2.imshow('dilate', dilate)
    cv2.imshow('result', result)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return data
text  = micr_number (outfile)
text1 = text

c000001c 40048504502 011825c 31



In [9]:
text = text.replace(" ", "") 
text = str(text)
print(text)
# text = text[-2]
# rbi_number = text[-8:-2]
# micr_number = str(text[-17:-8])
# print(routing_number)
# print(text)
# print(rbi_number)
# print(micr_number)

c000001c40048504502011825c31



In [10]:
check_number = text[0:8]
print(check_number)

c000001c


In [11]:
micr_number = text[8:19]
print(micr_number)

40048504502


In [12]:
rbi_number = text[19:26]
print(rbi_number)

011825c


In [13]:
routing_number = text[26:28]
print(routing_number)

31


In [14]:
text1= text1.replace("\n", " ")
text1= text1.replace("\x0c", "")
text1 = text1.split(" ")

In [15]:
text1

['c000001c', '40048504502', '011825c', '31', '']